In [1]:
%%capture
!pip install langchain

In [2]:
%%capture
!pip install huggingface_hub

In [3]:
%%capture
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "HuggingFaceH4/zephyr-7b-beta"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_name,quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [11]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=500,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [12]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7d6653b46c80>)

In [7]:
# https://huggingface.co/google/flan-t5-xl
# llm = HuggingFaceHub(repo_id="HuggingFaceH4/zephyr-7b-beta")

In [8]:
%%capture
!pip -q install langchain openai tiktoken chromadb

In [9]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
loader = DirectoryLoader('/content/data/', glob="./*.txt", loader_cls=TextLoader)

In [10]:
documents = loader.load()
documents

[Document(page_content='Welcome to Twixor EnCaps  \nWelcome! We are happy to have you here. To help you to use the Twixor EnCaps \nPlatform, we’ve prepared this series of guides for you.  \nWhat is Twixor EnCaps?  \nTwix or EnCaps is a platform that makes it possible to extend your customer \ncommunication to higher grounds. With its user -friendly drag and drop Engagement \nCapsule designer, you can engage your customers with multi -channel marketing.  \nWhat can you do with Twix or EnCaps?  \nFor most of the businesses, marketing campaigns are the primary method for \ninteractin g with their marketplace to strengthen their position in the customer minds, \nand for acquiring new customers and re -engaging their exi sting customers.  \nWhile mos t of the organizations can’t spend multi -million funds to promote their \nproducts, they can make powerful and unforgettable campaigns and broadcast them \nacross various channels like SMS, Twitter, Whatsapp, Messenger, etc.  \nUsing Twi xor, 

In [13]:
print(documents[0].page_content)

Welcome to Twixor EnCaps  
Welcome! We are happy to have you here. To help you to use the Twixor EnCaps 
Platform, we’ve prepared this series of guides for you.  
What is Twixor EnCaps?  
Twix or EnCaps is a platform that makes it possible to extend your customer 
communication to higher grounds. With its user -friendly drag and drop Engagement 
Capsule designer, you can engage your customers with multi -channel marketing.  
What can you do with Twix or EnCaps?  
For most of the businesses, marketing campaigns are the primary method for 
interactin g with their marketplace to strengthen their position in the customer minds, 
and for acquiring new customers and re -engaging their exi sting customers.  
While mos t of the organizations can’t spend multi -million funds to promote their 
products, they can make powerful and unforgettable campaigns and broadcast them 
across various channels like SMS, Twitter, Whatsapp, Messenger, etc.  
Using Twi xor, EnCaps you can design creative campaig

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
texts

[Document(page_content='Welcome to Twixor EnCaps  \nWelcome! We are happy to have you here. To help you to use the Twixor EnCaps \nPlatform, we’ve prepared this series of guides for you.  \nWhat is Twixor EnCaps?  \nTwix or EnCaps is a platform that makes it possible to extend your customer \ncommunication to higher grounds. With its user -friendly drag and drop Engagement \nCapsule designer, you can engage your customers with multi -channel marketing.  \nWhat can you do with Twix or EnCaps?', metadata={'source': '/content/data/twixor_data.txt'}),
 Document(page_content='What can you do with Twix or EnCaps?  \nFor most of the businesses, marketing campaigns are the primary method for \ninteractin g with their marketplace to strengthen their position in the customer minds, \nand for acquiring new customers and re -engaging their exi sting customers.  \nWhile mos t of the organizations can’t spend multi -million funds to promote their \nproducts, they can make powerful and unforgettable 

In [15]:
%%capture
!pip -q install langchain
!pip install pypdf
!pip install sentence-transformers==2.2.2

# !pip install openai
!pip install tiktoken
!pip install faiss-cpu

In [16]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [17]:
from langchain.vectorstores import FAISS

vectorstore=FAISS.from_documents(texts, embeddings)

persist_directory = 'db'
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)

In [18]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [19]:
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embeddings)

retriever = vectordb.as_retriever(search_kwargs={"k":3})
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7d66428d01c0>, search_kwargs={'k': 3})

In [20]:
# template="""You are an assistant for question-answering tasks.
# Use the following pieces of retrieved context to answer the question.
# If you don't know the answer, just say that you don't know.
# Use ten sentences maximum and keep the answer concise.
# Question: {question}
# Context: {context}
# Answer:
# """

# template="""You are an assistant for question-answering tasks.Don't cut the sentence halfly complete the answer fully.
# Given the information contained within the uploaded documents, provide a comprehensive answer to the question and be no longer than 500 words.
# If the answer to the question is found within the uploaded documents, also report the source URL from where the answer was derived.
# If the answer cannot be deduced from the information within the uploaded documents, do not provide an answer. Emphasize that no answers should be given based on knowledge outside of the provided documents,
# and indicate that the AI is trained exclusively on the content of the uploaded documents for the purpose of this task.</s>
# If the question ask about you please response I am a SF GenAI Assistant and help you to answer your question based on the uploaded documents.
# Question: {question}
# Context: {context}
# Answer:
# """

template = """
As an assistant specializing in question-answering tasks, it is crucial to ensure responses are well-structured and complete. When addressing the query provided, it is imperative to deliver a comprehensive response within a 500-word limit. Please adhere to the following guidelines:

1. If the information necessary to answer the question is available within the documents you have uploaded, please provide a detailed answer. Additionally, cite the source URL from which the answer was derived.
2. If the uploaded documents do not contain the information required to answer the question, clearly state that an answer cannot be provided. Emphasize that this decision is based on the absence of relevant information in the provided documents.
3. It is important to avoid basing your answer on any knowledge external to the content of the uploaded documents. Your responses should be solely informed by the information contained within these documents.
4. In instances where the question pertains to you, please respond with: "I am a SF GenAI Assistant, designed to help you answer questions based on the uploaded documents."

Following these instructions, please see below for the task at hand:

Question: {question}
Context: {context}
Answer:
"""


In [21]:
from langchain.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(template)

In [22]:
print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nAs an assistant specializing in question-answering tasks, it is crucial to ensure responses are well-structured and complete. When addressing the query provided, it is imperative to deliver a comprehensive response within a 500-word limit. Please adhere to the following guidelines:\n\n1. If the information necessary to answer the question is available within the documents you have uploaded, please provide a detailed answer. Additionally, cite the source URL from which the answer was derived.\n2. If the uploaded documents do not contain the information required to answer the question, clearly state that an answer cannot be provided. Emphasize that this decision is based on the absence of relevant information in the provided documents.\n3. It is important to avoid basing your answer on any knowledge external to the content of the uploaded

In [23]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [24]:
output_parser = StrOutputParser()

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

In [ ]:
from langchain.chains import LLMChain

In [ ]:
# Creating an LLM Chain
# llm_chain = LLMChain(llm=llm, prompt=prompt,memory=memory)
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

# # RAG Chain
# rag_chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | llm_chain
# )

In [ ]:
rag_chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7899421dc2e0>, search_kwargs={'k': 3}),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nAs an assistant specializing in question-answering tasks, it is crucial to ensure responses are well-structured and complete. When addressing the query provided, it is imperative to deliver a comprehensive response within a 500-word limit. Please adhere to the following guidelines:\n\n1. If the information necessary to answer the question is available within the documents you have uploaded, please provide a detailed answer. Additionally, cite the source URL from which the answer was derived.\n2. If the uploaded documents do not contain the information required to answer the question, clearly sta

In [ ]:
query = "What is Twixor Encaps?"
result=rag_chain.invoke(query)
print(result)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Human: 
As an assistant specializing in question-answering tasks, it is crucial to ensure responses are well-structured and complete. When addressing the query provided, it is imperative to deliver a comprehensive response within a 500-word limit. Please adhere to the following guidelines:

1. If the information necessary to answer the question is available within the documents you have uploaded, please provide a detailed answer. Additionally, cite the source URL from which the answer was derived.
2. If the uploaded documents do not contain the information required to answer the question, clearly state that an answer cannot be provided. Emphasize that this decision is based on the absence of relevant information in the provided documents.
3. It is important to avoid basing your answer on any knowledge external to the content of the uploaded documents. Your responses should be solely informed by the information contained within these documents.
4. In instances where the question pertain

In [ ]:
print(result.split("Answer:")[-1])


Based on the provided documents, Twixor Encaps is a platform that enables multi-channel marketing and allows for the extension of customer communication to higher grounds. It has a user-friendly drag and drop Engagement Capsule designer and falls under the category of actionable instant messaging (AIM) platforms that can be integrated into or embedded within existing IT applications. More specific details about its features and capabilities can be found through the FAQs section provided in one of the documents. The source for all of this information is "/content/data/twixor_data.txt".


In [ ]:
query = "Can you summarize this document"
result=rag_chain.invoke(query)
print(result.split("Answer:")[-1])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



This document provides instructions for attaching media, documents, messages, and maps to transform customer conversations into rich and interactive experiences. Users can add these artifacts in the Departments page by clicking the attachment icon and selecting the relevant file from their local device using the open dialogue box. Canned messages can also be preset and sent directly from the interface. The document includes a step-by-step guide for adding messages and documents, with details such as the date of creation and user information displayed once the message or document is saved.


In [ ]:
query = "how many question i have asked to you?"
result=rag_chain.invoke(query)
print(result.split("Answer:")[-1])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Based on the provided documents, I am unable to determine how many questions you have asked me. These documents appear to contain information related to customer interactions through WhatsApp and user management in Twixor, but they do not include any records of questions being asked to an AI assistant. Therefore, I must conclude that there is no relevant data to answer this particular query.


In [ ]:
query = "who are you"
result=rag_chain.invoke(query)
print(result.split("Answer:")[-1])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Based on the provided context, it is clear that the SF GenAI Assistant is not directly involved in the conversations or interactions described in the documents. Therefore, I am unable to provide information about my identity beyond what has already been communicated. However, if you require further clarification regarding the role and capabilities of the SF GenAI Assistant, please let us know.

In terms of answering the original question, the SF GenAI Assistant is a specialized artificial intelligence system designed to assist users in answering questions based on the content of uploaded documents. Its primary function is to process and analyze large volumes of textual data, identify relevant information, and provide accurate and insightful responses. The SF GenAI Assistant is not a physical entity but rather a virtual tool that can be accessed through various platforms and interfaces. Its performance is continually improving as it learns from new inputs and feedback, making it a valu

In [ ]:
from langchain.chains import ConversationalRetrievalChain

In [ ]:
from langchain.memory import ConversationBufferMemory

In [25]:
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [26]:
message_history = ChatMessageHistory()
memory = ConversationBufferMemory(
    memory_key="chat_history",
    output_key="answer",
    chat_memory=message_history,
    return_messages=True,
)

In [27]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7d6653b46c80>)

In [45]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder,HumanMessagePromptTemplate

In [128]:
chat_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=False)
# prompt = ChatPromptTemplate.from_messages([
#     SystemMessagePromptTemplate.from_template(
#         """You are an assistant for question-answering tasks.Don't cut the sentence halfly complete the answer fully.
# Given the information contained within the uploaded documents, provide a comprehensive answer to the question and be no longer than 500 words.
# If the answer to the question is found within the uploaded documents, also report the source URL from where the answer was derived.
# If the answer cannot be deduced from the information within the uploaded documents, do not provide an answer. Emphasize that no answers should be given based on knowledge outside of the provided documents,
# and indicate that the AI is trained exclusively on the content of the uploaded documents for the purpose of this task.</s>
# If the question ask about you please response I am a SF GenAI Assistant and help you to answer your question based on the uploaded documents.
# """),

#     MessagesPlaceholder(variable_name="chat_history"),
#     HumanMessagePromptTemplate.from_template("{question}")
# ])

template="""You are an assistant for question-answering tasks.Don't cut the sentence halfly complete the answer fully.
Given the information contained within the uploaded documents, provide a comprehensive answer to the question and be no longer than 500 words.
If the answer to the question is found within the uploaded documents, also report the source URL from where the answer was derived.
If the answer cannot be deduced from the information within the uploaded documents, do not provide an answer. Emphasize that no answers should be given based on knowledge outside of the provided documents,
and indicate that the AI is trained exclusively on the content of the uploaded documents for the purpose of this task.</s>
If the question ask about you please response I am a SF GenAI Assistant and help you to answer your question based on the uploaded documents.
Question: {question}
Context: {context}
Answer:
"""
prompt=ChatPromptTemplate.from_template(template)

In [129]:
# chain = ConversationalRetrievalChain.from_llm(
#     llm,
#     chain_type="stuff",
#     retriever=retriever,
#     memory=memory,
#     return_source_documents=False,
#     combine_docs_chain_kwargs={"prompt": qa_prompt}
# )

support_qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    verbose=False, condense_question_prompt=prompt,memory = chat_memory
)

In [130]:
query = "What is Twixor"
result = support_qa({"question": query})

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [131]:
print(result)

{'question': 'What is Twixor', 'chat_history': '', 'answer': 'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nTwixor EnCaps embodies standard support. You can contact Twixor support by clicking \nContact Us  in the webs ite for getting assistance on:  \n• Buying or Upgrading to Early  Adopter/ Volume User/ Enterprise User package  \n• Enquiry for On Premise/ Embedded option  \n• Installing and using Twixor EnCaps  \n• Pricing and Featu re Classification  \n• Identifying and verifying the causes of  suspected errors  \n• Others  \nFAQ\n\ncustomers and conversations. Twixor\'s chat functionality aids you to solve cases better.  \nThe interface is designed in such a way that it allows you to chat with several cu stomers \neven if you are bus y with other activities. Your work can be made easier  using Artificial \nIntelligence -powered with actionable click buttons for 

In [132]:
print(result['answer'].split("Helpful Answer:")[-1])

 Yes, Twixor EnCaps offers pre-built connectors to popular CRM systems like Salesforce, Microsoft Dynamics, and Zoho CRM


In [133]:
query = "What is Satisfation Rating?"
result = support_qa({"question": query})

ValueError: Unsupported chat history format: <class 'str'>. Full chat history: Human: What is Twixor
AI: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Twixor EnCaps embodies standard support. You can contact Twixor support by clicking 
Contact Us  in the webs ite for getting assistance on:  
• Buying or Upgrading to Early  Adopter/ Volume User/ Enterprise User package  
• Enquiry for On Premise/ Embedded option  
• Installing and using Twixor EnCaps  
• Pricing and Featu re Classification  
• Identifying and verifying the causes of  suspected errors  
• Others  
FAQ

customers and conversations. Twixor's chat functionality aids you to solve cases better.  
The interface is designed in such a way that it allows you to chat with several cu stomers 
even if you are bus y with other activities. Your work can be made easier  using Artificial 
Intelligence -powered with actio